In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json

In [2]:
ptt_url = ("https://www.ptt.cc")
ptt_movie_url = ptt_url+"/bbs/movie"
res = requests.get(ptt_movie_url)

In [3]:
soup = BeautifulSoup(res.text)
previous_page_url = ptt_url+soup.select(".btn.wide")[1]["href"]

movies = []
break_day = datetime.strptime("2018-01-01", '%Y-%m-%d')
check_date = datetime.strptime("2018-12-31", '%Y-%m-%d')
while break_day<check_date:
    movies_data = soup.select('.r-ent')
#     print(previous_page_url)
    for i in range(len(movies_data)):
        try:
            check_date_res = requests.get(ptt_url+movies_data[0].select(".title")[0].select("a")[0]["href"])
            check_date = BeautifulSoup(check_date_res.text).select(".article-meta-value")[-1].text
            check_date = datetime.strptime(check_date, '%a %b %d %H:%M:%S %Y')
            break
        except:
            continue

    for item in movies_data:
        movie = {}
        movie['title'] = item.select(".title")[0].text.strip()
        movie['date'] = item.select(".date")[0].text.strip()
        movie['author'] = item.select(".author")[0].text.strip()
        if item.select(".nrec")[0].text.strip()== '':
            movie['push'] = '0'
        else:
            movie['push'] = item.select(".nrec")[0].text.strip()
        movies.append(movie)
    soup = BeautifulSoup(requests.get(previous_page_url).text)
    previous_page_url = ptt_url+soup.select(".btn.wide")[1]["href"]
movies_json = json.dumps(movies)

In [4]:
with open('movie.json', 'w') as outfile:
    json.dump(movies, outfile)